<a href="https://colab.research.google.com/github/leonardo3108/robustez-query/blob/main/code/All_queries%2BBM25%2BRerank%2BnDCG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação

## Carga dos pacotes pygaggle e pyserini

In [1]:
!pip install git+https://github.com/castorini/pygaggle.git

  Cloning https://github.com/castorini/pygaggle.git to /tmp/pip-req-build-kqtr82tl
  Running command git clone -q https://github.com/castorini/pygaggle.git /tmp/pip-req-build-kqtr82tl
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 43 kB 1.0 MB/s 
     |████████████████████████████████| 9.1 MB 6.9 MB/s 
     |████████████████████████████████| 75.5 MB 4.0 kB/s 
     |████████████████████████████████| 22.3 MB 1.6 MB/s 
     |████████████████████████████████| 25.9 MB 1.4 MB/s 
     |████████████████████████████████| 12.8 MB 21.0 MB/s 
     |████████████████████████████████| 6.0 MB 22.2 MB/s 
     |████████████████████████████████| 454.3 MB 18 kB/s 
     |████████████████████████████████| 3.3 MB 39.5 MB/s 
     |████████████████████████████████| 72 kB 1.0 MB/s 
     |████████████████████████████████| 2.2 MB 38.5 MB/s 
     |████████████████████████████████| 1.2 MB 33.6 MB/s 
     |████████████████████████████████| 85 kB 4.4 MB/s 
     |█

In [2]:
!pip install pyserini

## Carga do pacote Faiss (GPU)

In [4]:
!pip install faiss-gpu

     |████████████████████████████████| 89.7 MB 9.5 kB/s 


## Carga do dataset MsMarco Passage

In [5]:
from pyserini.search import SimpleSearcher

# `SimpleSearcher` defaults to BM25 scoring function.
searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')

index-msmarco-passage-20201117-f87c94.tar.gz: 0.00B [00:00, ?B/s]

Attempting to initialize pre-built index msmarco-passage.


index-msmarco-passage-20201117-f87c94.tar.gz: 2.07GB [02:44, 13.5MB/s]                            


Extracting /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.tar.gz into /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d...
Initializing msmarco-passage...


## Carga das queries originais

In [6]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/noisy-queries.csv

--2021-11-18 19:13:11--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/noisy-queries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4550 (4.4K) [text/plain]
Saving to: ‘noisy-queries.csv’

noisy-queries.csv   100%[===================>]   4.44K  --.-KB/s    in 0s      

2021-11-18 19:13:11 (40.7 MB/s) - ‘noisy-queries.csv’ saved [4550/4550]



In [7]:
import pandas as pd

df = pd.read_csv('noisy-queries.csv')
df.set_index('Query#', inplace=True)
df

,original,noisy-translation-pt
Query#,,
23849,are naturalization records public information,is public information from naturalization reco...
42255,average salary for dental hygienist in nebraska,Average Salary of Dental Hygienist in Nebraska
47210,average wedding dress alteration cost,average cost of changing the wedding dress
67316,can fever cause miscarriage early pregnancy,can fever cause miscarriage in early pregnancy
118440,define bmt medical,define bmt medical
121171,define etruscans,define Etruscans
135802,definition of laudable,definition of commendable
141630,describe how muscles and bones work together t...,describe how muscles and bones work together t...
156498,do google docs auto save,Google docs automatically saves


## Carga dos julgamentos do TREC 2020


In [8]:
! wget -nc https://trec.nist.gov/data/deep/2020qrels-pass.txt

--2021-11-18 19:13:25--  https://trec.nist.gov/data/deep/2020qrels-pass.txt
Resolving trec.nist.gov (trec.nist.gov)... 132.163.4.175, 2610:20:6005:13::19
Connecting to trec.nist.gov (trec.nist.gov)|132.163.4.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 218617 (213K) [text/plain]
Saving to: ‘2020qrels-pass.txt’

2020qrels-pass.txt  100%[===================>] 213.49K   374KB/s    in 0.6s    

2021-11-18 19:13:26 (374 KB/s) - ‘2020qrels-pass.txt’ saved [218617/218617]



In [9]:
judgment = {}
for i, line in enumerate(open('2020qrels-pass.txt')):
    query_nr, _, pid, eval = line.rstrip().split()
    judgment[(int(query_nr), pid)] = int(eval)

In [10]:
scale = {3:'perfectly relevant', 2:'highly relevant', 1:'related', 0:'Irrelevant'}

## Carga do Reranker MonoT5

In [11]:
!nvidia-smi

Thu Nov 18 19:13:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
from pygaggle.rerank.transformer import MonoT5

reranker = MonoT5() 

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

# Execução

## Rank BM25 (1000 melhores ranks)

In [17]:
import json
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.base import hits_to_texts

query = {}
texts = {}

print('BM25:')
for query_number, row in df.iterrows():
    for noise in df.columns:
        query_text = row[noise]
        print(query_number, query_text + ' (' + noise + '):')
        query[(query_number, noise)] = Query(query_text)
        hits = searcher.search(query[(query_number, noise)].text, k=1000)
        texts[(query_number, noise)] = hits_to_texts(hits)
        for i, doc in enumerate(texts[(query_number, noise)][:10]):
            content = json.loads(doc.text)['contents']
            print(f'\t{i+1:2} {doc.metadata["docid"]:15} {doc.score:.5f} {content}')

BM25:
23849 are naturalization records public information (original):
	 1 4348282         10.06630 Civil Records Definition. Civil records are a group of public records that pertain to civil registry records, civil family matters and non criminal civil offenses. These records vary a lot because of the nature of the information that is recorded.
	 2 2674124         9.86550 See our FAQ's to learn how you can benefit by using our search. Share: DISCLAIMER: Due to the nature of the origin of public record information, the public records and commercially available data sources used in the PeopleWise system may contain errors.
	 3 7119957         9.64420 Yes, in most cases Public Records are available to the public. Some documents, such as certain court records, confidential personal information, and other sensitive information may be kept sealed or is only available with a court order. In certain states, there is a waiting period to obtain Public Records that reveal private information. Whi

## Rerank

In [18]:
reranked = {}
results = {}

print('Reranked:')
for query_number, row in df.iterrows():
    for noise in df.columns:
        query_text = row[noise]
        print(query_number, query_text + ' (' + noise + '):')
        reranked[(query_number, noise)] = reranker.rerank(query[(query_number, noise)], texts[(query_number, noise)])
        results[(query_number, noise)] = []

        for i, doc in enumerate(reranked[(query_number, noise)][:10]):
            results[(query_number, noise)].append(doc.metadata["docid"])
            content = json.loads(doc.text)['contents']
            print(f'\t{i+1:2} {doc.metadata["docid"]:15} {doc.score:.5f}    {content}')

Reranked:
23849 are naturalization records public information (original):
	 1 2647769         -0.00595    Naturalization Record Search The Naturalization and Petition of Record books are available in the Prothonotary Office for public viewing.
	 2 8010561         -0.01348    Department of State >> Delaware Public Archives. Naturalization Records Database. Naturalization records can be found in a number of series amongst the holdings of the Delaware Public Archives. The earliest material can be found in the records of the Court of Common Pleas and the Supreme Court of each county.
	 3 8010558         -0.02736    Pre-1906 naturalization records (whether they are certificates of naturalizations, declarations of intention, petitions, or oaths and whether the naturalization was granted in state or federal court) provide far less information about the applicant than do the post-1906 records.
	 4 8059826         -0.12631    Are all types of Public Records open to the public? Yes, in most case

# Avaliação

## Batimento com os julgamentos

In [19]:
for query_number, row in df.iterrows():
    for noise in df.columns:
        query_text = row[noise]
        print(query_number, query_text + ' (' + noise + '):')
        for i, doc in enumerate(reranked[(query_number, noise)][:10]):
            content = json.loads(doc.text)['contents']
            eval = judgment.get((query_number, doc.metadata["docid"]), 0)
            print(f'\t{i+1:2} {doc.metadata["docid"]:15} {doc.score:.5f}\t{eval} ({scale[eval]:18})    {content}')

23849 are naturalization records public information (original):
	 1 2647769         -0.00595	3 (perfectly relevant)    Naturalization Record Search The Naturalization and Petition of Record books are available in the Prothonotary Office for public viewing.
	 2 8010561         -0.01348	3 (perfectly relevant)    Department of State >> Delaware Public Archives. Naturalization Records Database. Naturalization records can be found in a number of series amongst the holdings of the Delaware Public Archives. The earliest material can be found in the records of the Court of Common Pleas and the Supreme Court of each county.
	 3 8010558         -0.02736	2 (highly relevant   )    Pre-1906 naturalization records (whether they are certificates of naturalizations, declarations of intention, petitions, or oaths and whether the naturalization was granted in state or federal court) provide far less information about the applicant than do the post-1906 records.
	 4 8059826         -0.12631	1 (related   

## Obtenção do IDCG

In [22]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/idcg.csv

--2021-11-18 20:02:24--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/idcg.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1467 (1.4K) [text/plain]
Saving to: ‘idcg.csv’

idcg.csv            100%[===================>]   1.43K  --.-KB/s    in 0s      

2021-11-18 20:02:24 (23.9 MB/s) - ‘idcg.csv’ saved [1467/1467]



In [23]:
df_IDCG = pd.read_csv('idcg.csv')
df_IDCG.rename(columns={'query': 'Query#', ' idcg': 'IDCG@10'}, inplace=True)
df_IDCG.set_index('Query#', inplace=True)
df_IDCG

,IDCG@10
Query#,
23849,31.804915
42255,18.771051
47210,28.182677
67316,26.849343
118440,31.804915
121171,13.630678
135802,13.052548
141630,20.154397
156498,15.819559


## Cálculo do DCG@10

In [24]:
import math

dcg10 = {}

for query, noise in results.keys():
    acumulated = 0
    for i, doc in enumerate(results[(query, noise)]):
        eval = judgment.get((query, doc), 0)
        acumulated += (2**int(eval)-1) / math.log2(i+2)
    if query not in dcg10:
        dcg10[query] = {}
    dcg10[query][noise] = acumulated

In [26]:
df_DCG10 = pd.DataFrame.from_dict(dcg10).T
df_DCG10.index.name = 'Query#'
df_DCG10['IDCG@10'] = df_IDCG['IDCG@10']
df_DCG10['Text'] = df['original']
df_DCG10

,original,noisy-translation-pt,IDCG@10,Text
Query#,,,,
23849,16.689854,11.771240,31.804915,are naturalization records public information
42255,17.075732,17.277160,18.771051,average salary for dental hygienist in nebraska
47210,26.772613,26.218413,28.182677,average wedding dress alteration cost
67316,17.369096,17.369096,26.849343,can fever cause miscarriage early pregnancy
118440,8.857052,8.857052,31.804915,define bmt medical
121171,6.153273,5.961826,13.630678,define etruscans
135802,9.850383,0.630930,13.052548,definition of laudable
141630,12.465214,12.465214,20.154397,describe how muscles and bones work together t...
156498,11.396068,12.271201,15.819559,do google docs auto save


In [27]:
import csv

df_DCG10.to_csv('dcg10_rerank.csv', header=True, index=True, index_label='Query#', quoting = csv.QUOTE_NONNUMERIC)

## Cálculo do nDCG@10

In [28]:
df_nDCG10 = pd.DataFrame(index=df_DCG10.index)
for column in list(df_DCG10.columns)[:-2]:
    df_nDCG10[column] = df_DCG10[column] / df_DCG10['IDCG@10']
df_nDCG10['Text'] = df['original']
df_nDCG10

2021-11-18 20:03:27 [INFO] utils: NumExpr defaulting to 2 threads.


,original,noisy-translation-pt,Text
Query#,,,
23849,0.524757,0.370108,are naturalization records public information
42255,0.909684,0.920415,average salary for dental hygienist in nebraska
47210,0.949967,0.930302,average wedding dress alteration cost
67316,0.646910,0.646910,can fever cause miscarriage early pregnancy
118440,0.278481,0.278481,define bmt medical
121171,0.451428,0.437383,define etruscans
135802,0.754671,0.048338,definition of laudable
141630,0.618486,0.618486,describe how muscles and bones work together t...
156498,0.720378,0.775698,do google docs auto save


In [29]:
df_nDCG10.to_csv('ndcg10_rerank.csv', header=True, index=True, index_label='Query#', quoting = csv.QUOTE_NONNUMERIC)

In [30]:
df_nDCG10.describe()

,original,noisy-translation-pt
count,54.000000,54.000000
mean,0.669499,0.587796
std,0.226781,0.292799
min,0.050171,0.000000
25%,0.559207,0.404927
50%,0.695759,0.670329
75%,0.809561,0.789984
max,1.000000,1.000000
